<a href="https://colab.research.google.com/github/timwu64/Generative-AI--LangChainlangchain-Real-Estate-Listing-Geneation-and-Semantic-Search-for-Home-Match/blob/main/HomeMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Introduction:

As a talented developer at "Future Homes Realty", a forward-thinking real estate company. In an industry where personalization is key to customer satisfaction, the company wants to revolutionize how clients interact with real estate listings. The goal is to create a personalized experience for each buyer, making the property search process more engaging and tailored to individual preferences.
The Challenge:

The task is to develop an innovative application named "HomeMatch". This application leverages large language models (LLMs) and vector databases to transform standard real estate listings into personalized narratives that resonate with potential buyers' unique preferences and needs.

Step 1: Setting Up the Python Application

Initialize a Python Project: Create a new Python project, setting up a virtual environment and installing necessary packages like LangChain, a suitable LLM library (e.g., OpenAI's GPT), and a vector database package compatible with Python (e.g., ChromaDB or LanceDB). If you don't wish to create your files from scratch, starter files are available in the workspace on the next page as an application skeleton.

In [1]:
#!pip install langchain==0.0.305
#!pip install openai==0.28.1
#!pip install chromadb==0.4.15
#!pip install tiktoken
#!pip install langchain_core

In [2]:
from google.colab import drive
drive.mount('/content/drive')

project_file_path = '/content/drive/MyDrive/Code/Udacity/Generative_AI/Project-3_Personalized_Real_Estate_Agent/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import openai
from langchain import LLMChain
from langchain.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains.question_answering import load_qa_chain
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import pandas as pd
from tqdm import tqdm
import random

print(openai.__version__)

0.28.1


In [4]:
# Path to the file containing the API key
api_key_file = project_file_path+'api_key.txt'

# Function to read the API key from the file
def get_api_key(file_path):
    with open(file_path, 'r') as file:
        return file.read().strip()  # .strip() removes any leading/trailing whitespace

# Retrieve the API key from the file
openai.api_key = get_api_key(api_key_file)
#display(openai.api_key)

In [5]:
model_name = "gpt-4"
temperature = 0.8
# Initialize the OpenAI LLM through LangChain
llm = OpenAI(openai_api_key=openai.api_key, model_name=model_name, temperature=temperature, max_tokens = 4000)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Step 2: Generating Real Estate Listings

Generate real estate listings using a Large Language Model. Generate at least 10 listings This can involve creating prompts for the LLM to produce descriptions of various properties. An example of a listing might be:
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.

In [6]:
def parse_listing(text):
    """
    Parse the generated listing text into structured data.
    """
    # Split the text by lines and filter out empty lines
    lines = [line.strip() for line in text.split('\n') if line.strip()]

    # Initialize a dictionary to hold the parsed data
    listing_data = {}
    for line in lines:
        # Split each line by the first colon to separate the field name from the value
        parts = line.split(':', 1)
        if len(parts) == 2:
            field_name = parts[0].strip()
            field_value = parts[1].strip()
            listing_data[field_name] = field_value

    return listing_data

In [7]:
def generate_real_estate_listings(n=5):
    listings = []
    for i in tqdm(range(n), desc="Generating Listings"):
        # Generate unique listing number
        Unique_ID = f"L{random.randint(100000, 999999)}"

        # Attributes with variability
        neighborhood = random.choice(["Beverly Hills, California",
                                      "Tribeca, New York City",
                                      "Pacific Heights, San Francisco",
                                      "South Beach, Miami",
                                      "Georgetown, Washington, D.C.",
                                      "Back Bay, Boston",
                                      "The Hamptons, New York",
                                      "Buckhead, Atlanta",
                                      "River Oaks, Houston",
                                      "Old Town, Chicago"])
        price = random.randint(800000, 1200000)  # Price range specified
        bedrooms = random.choice([3, 4, 5])  # Allowing deviation
        bathrooms = random.choice([2, 3, 4])  # Allowing deviation
        house_size = random.choice([1500, 2000, 2500])  # Square feet, allowing deviation

        # Define a prompt for generating a listing
        data_gen_template = f"""You are a creative real estate agent, tasked generate a detailed real estate listing with Highlight unique features, Use positive, descriptive language, including Unique ID, Neighborhood, Price, Bedrooms, Bathrooms, House Size, Description, Neighborhood Description.
        The Description should have information regarding bedroom, bathroom, kitchen, living room, amenities, energy and heating system, garage, BBQ, backyard.
        The Neighborhood Description should cover good schools, convenient shopping options, transportation options, bus line, highway, and bike, urban, suburban, restaurants, theaters.
        An example of a listing might be---
        Unique ID: {Unique_ID}
        Neighborhood: {neighborhood}
        Price ($): {price}
        Bedrooms: {bedrooms}
        Bathrooms: {bathrooms}
        House Size (sqft): {house_size}
        Description: Welcome to this eco-friendly oasis nestled in the heart of {neighborhood}. This charming {bedrooms}-bedroom, {bathrooms}-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.
        Neighborhood Description: {neighborhood} is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.---
        """

        # Generate the listing
        data_gen_prompt = PromptTemplate.from_template(data_gen_template)
        response = llm(data_gen_prompt.format())

        # Parse the listing text into structured data
        listing_data = parse_listing(response)

        # Append the structured data to the listings list
        listings.append(listing_data)

    return listings

In [8]:
# Generate real estate listings
listings = generate_real_estate_listings(100)

# Create a DataFrame from the listings
df = pd.DataFrame(listings)

# Print the DataFrame to verify its contents
df.head(10)

Generating Listings: 100%|██████████| 100/100 [16:10<00:00,  9.71s/it]


,Unique ID,Neighborhood,Price ($),Bedrooms,Bathrooms,House Size (sqft),Description,Neighborhood Description
0,B792305,"Brentwood, Los Angeles","1,450,000",4,3,3000,"Welcome to this radiant Brentwood beauty, offe...","Brentwood, a wealthy district in Los Angeles, ..."
1,E528379,"Beacon Hill, Boston","1,275,000",4,3,2100,Introducing a charming and distinctive 4-bedro...,Beacon Hill is an idyllic and prosperous neigh...
2,H721436,"Hyde Park, Chicago",1195000,5,3.5,3500,Discover the epitome of urban sophistication w...,"Hyde Park, with its rich history and diverse c..."
3,E726593,"Eastlake, Seattle","1,450,000",4,3.5,2800,"Welcome to this striking 4-bedroom, 3.5-bathro...","Eastlake, Seattle is an urban neighborhood kno..."
4,L00001GB,"Green Belt, Austin","899,000",3,3.5,2800,Enter a world of elegance and class with this ...,"Green Belt, Austin is a vibrant and welcoming ..."
5,H956321,"Westlake, Seattle","1,150,000",4,3,3150,Welcome to a breathtakingly beautiful 4-bedroo...,"Westlake is a thriving community, beautifully ..."
6,R2020025,"Lakeview, Chicago","1,350,000",4,3,"3,000",Immerse yourself in the warmth and luxury of t...,"Situated in the vibrant Lakeview, Chicago, you..."
7,H123456,"South Lake Union, Seattle","1,200,000",3,3.5,2300,Welcome to the epitome of luxury and comfort c...,South Lake Union is a bustling urban neighborh...
8,H548241,"Bellevue, Washington","1,275,000",4,3,3200,"Step into this luxurious, meticulously designe...","Bellevue, often rated as one of the best place..."
9,R583261,"Capitol Hill, Seattle","1,350,800",5,3,2800,"Welcome to this architectural marvel, located ...","Capitol Hill, Seattle is renowned for its vibr..."


In [9]:
df['Price ($)'] = df['Price ($)'].str.replace(r'[$,]', '', regex=True).astype('int64')
df['Bedrooms'] = df['Bedrooms'].astype('int64')
df['Bathrooms'] = df['Bathrooms'].astype('float64')
df['House Size (sqft)'] = df['House Size (sqft)'].str.replace(r'[ sqft,]', '', regex=True).astype('int64')
print(df.dtypes)

Unique ID                    object
Neighborhood                 object
Price ($)                     int64
Bedrooms                      int64
Bathrooms                   float64
House Size (sqft)             int64
Description                  object
Neighborhood Description     object
dtype: object


In [10]:
print (df.shape)

# Drop rows with any empty cells
df.dropna(inplace=True)

# Remove duplicate records
df.drop_duplicates(inplace=True)

print (df.shape)

(100, 8)
(100, 8)


In [11]:
# Save the DataFrame to a CSV file

csv_file_path = project_file_path+"real_estate_listings.csv"
df.to_csv(csv_file_path, index=False)
print(f"Listings saved to {csv_file_path}")

Listings saved to /content/drive/MyDrive/Code/Udacity/Generative_AI/Project-3_Personalized_Real_Estate_Agent/real_estate_listings.csv


In [12]:
# Initialize the OpenAI LLM through LangChain

model_name = 'gpt-3.5-turbo'
temperature = 0

llm = OpenAI(openai_api_key=openai.api_key, model_name=model_name, temperature=temperature, max_tokens = 4000)

/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Load Real Estate Listings

In [27]:
# load the listing database from a csv file

df = pd.read_csv(project_file_path+"real_estate_listings.csv")

df.head()

,Unique ID,Neighborhood,Price ($),Bedrooms,Bathrooms,House Size (sqft),Description,Neighborhood Description
0,B792305,"Brentwood, Los Angeles",1450000,4,3.0,3000,"Welcome to this radiant Brentwood beauty, offe...","Brentwood, a wealthy district in Los Angeles, ..."
1,E528379,"Beacon Hill, Boston",1275000,4,3.0,2100,Introducing a charming and distinctive 4-bedro...,Beacon Hill is an idyllic and prosperous neigh...
2,H721436,"Hyde Park, Chicago",1195000,5,3.5,3500,Discover the epitome of urban sophistication w...,"Hyde Park, with its rich history and diverse c..."
3,E726593,"Eastlake, Seattle",1450000,4,3.5,2800,"Welcome to this striking 4-bedroom, 3.5-bathro...","Eastlake, Seattle is an urban neighborhood kno..."
4,L00001GB,"Green Belt, Austin",899000,3,3.5,2800,Enter a world of elegance and class with this ...,"Green Belt, Austin is a vibrant and welcoming ..."


Step 3: Building the User Preference Interface

Collect buyer preferences, such as the number of bedrooms, bathrooms, location, and other specific requirements from a set of questions or telling the buyer to enter their preferences in natural language. You can hard-code the buyer preferences in questions and answers, or collect them interactively however you'd like, example:

questions = [   
                "How big do you want your house to be?"
                "What are 3 most important things for you in choosing this property?",
                "Which amenities would you like?",
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]

answers = [
    "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
    "A quiet neighborhood, good local schools, and convenient shopping options.",
    "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
    "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
    "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
    ]

Buyer Preference Parsing: Implement logic to interpret and structure these preferences for querying the vector database.

In [28]:
def collect_buyer_preferences():
    questions = [
        "What is the range of your budget for buying a house? (e.g., 700000-3000000)",
        "What size range are you looking for in square feet for your house? (e.g., 1000-2500)",
        "What is the minimum number of Bedrooms you are looking for? (e.g., 3)",
        "What is the minimum number of Bathrooms you are looking for? (e.g., 3)",
        "What are 3 most important things for you in choosing this property? (e.g., A quiet neighborhood, good local schools, and convenient shopping options.)",
        "Which amenities would you like? (e.g., A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.)",
        "Which transportation options are important to you? (e.g., Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.)",
        "Do you have any other preferences? (e.g., A balance between suburban tranquility and urban amenities.)",
    ]

    # Initialize an empty list to hold each row (question and answer)
    data = []

    # Iterate through each question, collecting the answer
    for question in questions:
        print(question)  # Display the question
        answer = input()  # Collect the user's answer
        data.append({"Question": question, "Answer": answer})  # Append the question and answer as a dict

    # Convert the list of dicts into a DataFrame
    answers_df = pd.DataFrame(data)

    # Return the DataFrame containing the answers
    return answers_df

# Collect preferences from the user and store them in a DataFrame
buyer_preferences_df = collect_buyer_preferences()

What is the range of your budget for buying a house? (e.g., 700000-3000000)
700000-3000000
What size range are you looking for in square feet for your house? (e.g., 1000-2500)
1000-2500
What is the minimum number of Bedrooms you are looking for? (e.g., 3)
3
What is the minimum number of Bathrooms you are looking for? (e.g., 3)
3
What are 3 most important things for you in choosing this property? (e.g., A quiet neighborhood, good local schools, and convenient shopping options.)
A quiet neighborhood, good local schools, and convenient shopping options.
Which amenities would you like? (e.g., A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.)
A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
Which transportation options are important to you? (e.g., Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.)
Easy access to a reliable bus line, proximity to a major highway, and b

In [29]:
# Display the DataFrame to verify the collected answers
buyer_preferences_df.head(8)

,Question,Answer
0,What is the range of your budget for buying a ...,700000-3000000
1,What size range are you looking for in square ...,1000-2500
2,What is the minimum number of Bedrooms you are...,3
3,What is the minimum number of Bathrooms you ar...,3
4,What are 3 most important things for you in ch...,"A quiet neighborhood, good local schools, and ..."
5,"Which amenities would you like? (e.g., A backy...","A backyard for gardening, a two-car garage, an..."
6,Which transportation options are important to ...,"Easy access to a reliable bus line, proximity ..."
7,"Do you have any other preferences? (e.g., bala...",Balance between suburban tranquility and urban...


In [30]:
# Extracting specific answers
# Extracting budget range
budget_range = buyer_preferences_df["Answer"][0].split('-')
low_budget = int(budget_range[0])
high_budget = int(budget_range[1])

# Extracting house size range
size_range = buyer_preferences_df["Answer"][1].split('-')
small_house_size = int(size_range[0])
big_house_size = int(size_range[1])

minimum_bedrooms = int(buyer_preferences_df["Answer"][2])
minimum_bathrooms = float(buyer_preferences_df["Answer"][3])
important_factors = buyer_preferences_df["Answer"][4]
amenities = buyer_preferences_df["Answer"][5]
transportation_options = buyer_preferences_df["Answer"][6]
other_preference = buyer_preferences_df["Answer"][7]

customer_preference = important_factors + ' ' + amenities + ' ' + transportation_options + ' ' + other_preference

# Display extracted answers for verification
print(f"Budget Range: ${budget_range}")
print(f"House Size Range: {size_range} sqft")
print(f"Minimum Bedrooms: {minimum_bedrooms}")
print(f"Minimum Bathrooms: {minimum_bathrooms}")
print(f"Important Factors: {important_factors}")
print(f"Amenities: {amenities}")
print(f"Transportation Options: {transportation_options}")
print(f"Other Preference: {other_preference}")
print(f"Customer Preference: {customer_preference}")

Budget Range: $['700000', '3000000']
House Size Range: ['1000', '2500'] sqft
Minimum Bedrooms: 3
Minimum Bathrooms: 3.0
Important Factors: A quiet neighborhood, good local schools, and convenient shopping options.
Amenities: A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.
Transportation Options: Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.
Other Preference: Balance between suburban tranquility and urban amenities.
Customer Preference: A quiet neighborhood, good local schools, and convenient shopping options. A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system. Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads. Balance between suburban tranquility and urban amenities.


Step 4: Storing Listings in a Vector Database

Vector Database Setup: Initialize and configure ChromaDB or a similar vector database to store real estate listings.

Generating and Storing Embeddings: Convert the LLM-generated listings into suitable embeddings that capture the semantic content of each listing, and store these embeddings in the vector database.


In [31]:
df['Langchain_page_content'] = ('UID: ' + df["Unique ID"].astype(str) +
                                ' Neighborhood: ' + df["Neighborhood"].astype(str) +
                                ' Price: ' + df["Price ($)"].astype(str) +
                                ' Size: ' + df["House Size (sqft)"].astype(str) +
                                ' Bedrooms: ' + df["Bedrooms"].astype(str) +
                                ' Bathrooms: ' + df["Bathrooms"].astype(str) +
                                ', ' + df['Description'].astype(str) +
                                ' ' + df['Neighborhood Description'].astype(str))


df = df.drop(['Description', 'Neighborhood Description'], axis=1)

data_sort = df[ (df["Price ($)"] <= high_budget) & (df["Price ($)"] >= low_budget) &
                (df["House Size (sqft)"] <= big_house_size) & (df["House Size (sqft)"] >= small_house_size) &
                (df["Bedrooms"] >= minimum_bedrooms) &
                (df["Bathrooms"] >= minimum_bathrooms) ]

# Clearn the data frame by remove empty cells and duplicate records

# Drop rows with any empty cells
data_sort.dropna(inplace=True)

# Remove duplicate records
data_sort.drop_duplicates(inplace=True)

print (data_sort.shape)
data_sort.head()

(17, 7)


<ipython-input-31-e4ebc2bada1f>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sort.dropna(inplace=True)
<ipython-input-31-e4ebc2bada1f>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sort.drop_duplicates(inplace=True)


,Unique ID,Neighborhood,Price ($),Bedrooms,Bathrooms,House Size (sqft),Langchain_page_content
1,E528379,"Beacon Hill, Boston",1275000,4,3.0,2100,"UID: E528379 Neighborhood: Beacon Hill, Boston..."
7,H123456,"South Lake Union, Seattle",1200000,3,3.5,2300,"UID: H123456 Neighborhood: South Lake Union, S..."
15,L101938,"Beacon Hill, Boston",1200000,4,3.0,2500,"UID: L101938 Neighborhood: Beacon Hill, Boston..."
31,P786542,"Walnut Creek, California",1378000,4,3.0,2100,"UID: P786542 Neighborhood: Walnut Creek, Calif..."
32,H789324,"East Village, New York City",2345000,3,3.0,2200,"UID: H789324 Neighborhood: East Village, New Y..."


In [32]:
# Save the DataFrame to a CSV file

csv_file_path = project_file_path+"sorted_real_estate_listings.csv"
data_sort.to_csv(csv_file_path, index=False)
print(f"Listings saved to {csv_file_path}")

Listings saved to /content/drive/MyDrive/Code/Udacity/Generative_AI/Project-3_Personalized_Real_Estate_Agent/sorted_real_estate_listings.csv


In [33]:
loader = DataFrameLoader(data_sort, page_content_column="Langchain_page_content")
docs = loader.load()
#display(docs)

In [34]:
#loader = CSVLoader(file_path=project_file_path+"sorted_real_estate_listings.csv")
#docs = loader.load()
#display(docs)

In [35]:
splitter = CharacterTextSplitter(chunk_size=2000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(openai_api_key=openai.api_key)

db = Chroma.from_documents(split_docs, embeddings)
retriever=db.as_retriever()

In [36]:
query = f"Retrieve the home listings that match {customer_preference}."

In [37]:
prompt = PromptTemplate(
        template = (
                    "You are a professional real estate agent assisting home buyers. "
                    "Based on the provided Listings, identify the ones that best align with the customer's perference. "
                    "You may recommend up to three properties from the Listings in your response. "
                    "Begin your Answer on a new line with the statement: "
                    "'The home(s) that best meet your preferences are: ' "
                    "Each suggested property should be listed on a new line, including all required metadata: "
                    "'UID', 'Neighborhood', 'Price', 'Size', 'Bedrooms', 'Bathrooms', with no omissions. "
                    "Follow each listing with a Description that highlights features relevant to the buyer's needs, "
                    "Make sure you do not paraphrase the Description, and only use the information provided in the Description, no fabrications are permitted. "
                    "Preferences: {query} "
                    "Listings: {context} "
                    "Answer:"
                    ),
        input_variables=["query", "context"],
    )
print(prompt)

input_variables=['query', 'context'] template="You are a professional real estate agent assisting home buyers. Based on the provided Listings, identify the ones that best align with the customer's perference. You may recommend up to three properties from the Listings in your response. Begin your Answer on a new line with the statement: 'The home(s) that best meet your preferences are: ' Each suggested property should be listed on a new line, including all required metadata: 'UID', 'Neighborhood', 'Price', 'Size', 'Bedrooms', 'Bathrooms', with no omissions. Follow each listing with a Description that highlights features relevant to the buyer's needs, Make sure you do not paraphrase the Description, and only use the information provided in the Description, no fabrications are permitted. Preferences: {query} Listings: {context} Answer:"


In [38]:
similar_docs = db.similarity_search(query, k=10)
#similar_docs = db.similarity_search_with_score(query)
#display(similar_docs)

In [39]:
# Load the QA chain
chain = load_qa_chain(llm, prompt=prompt, chain_type="stuff")

# Run the chain with the extracted Document objects and the query
results = chain.run(input_documents=similar_docs, query=query)

print(results)

The home(s) that best meet your preferences are: 

UID: P981245 
Neighborhood: North Boulder, Colorado 
Price: 875000 
Size: 2300 
Bedrooms: 4 
Bathrooms: 3.0 

Description: Welcome to a delightful blend of comfort and elegance in this 4-bedroom, 3-bathroom jewel nestled in the heart of North Boulder. The definitive open-plan kitchen features state-of-the-art stainless steel appliances and stunning granite countertops, while the spacious living room, warmed by the cozy gas fireplace, provides panoramic views of the lush, landscaped backyard. All bedrooms are generously proportioned, with the master suite offering a walk-in closet and an opulent en-suite bathroom. The fully fenced backyard is an entertainer's dream, complete with a built-in BBQ, beautiful patio furniture, and plenty of room for outdoor activities. This home also features a two-car attached garage, high-efficiency heating system, and up-to-date insulation for energy conservation. North Boulder is an active, family-friend